In [2]:
import numpy as np
import torch
import torch_geometric
from torch_cluster import knn, knn_graph
import open3d as o3d
#utils
import h5py
import os
import dataclasses

from tqdm.notebook import tqdm, trange


ModuleNotFoundError: No module named 'pptk'

In [2]:
pip install pptk

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pptk (from versions: none)
ERROR: No matching distribution found for pptk


## Utils

In [ ]:
v = p

### Load data

In [2]:
def load_h5_data(h5_filename='', opts=None, skip_rate = 1, use_randominput=True):
    """
    skip_rate: {int} -- step_size when loading the dataset
    """
    num_point = opts.num_point
    num_4X_point = int(opts.num_point*4)
    num_out_point = int(opts.num_point*opts.up_ratio)

    print("h5_filename : ",h5_filename)
    if use_randominput:
        print("use randominput, input h5 file is:", h5_filename)
        f = h5py.File(h5_filename)
        input = f['poisson_%d'%num_4X_point][:]
        gt = f['poisson_%d'%num_out_point][:]
    else:
        print("Do not randominput, input h5 file is:", h5_filename)
        f = h5py.File(h5_filename)
        input = f['poisson_%d' % num_point][:]
        gt = f['poisson_%d' % num_out_point][:]

    #name = f['name'][:]
    assert len(input) == len(gt)

    print("Normalization the data")
    data_radius = np.ones(shape=(len(input)))
    centroid = np.mean(gt[:, :, 0:3], axis=1, keepdims=True)
    gt[:, :, 0:3] = gt[:, :, 0:3] - centroid
    furthest_distance = np.amax(np.sqrt(np.sum(gt[:, :, 0:3] ** 2, axis=-1)), axis=1, keepdims=True)
    gt[:, :, 0:3] = gt[:, :, 0:3] / np.expand_dims(furthest_distance, axis=-1)
    input[:, :, 0:3] = input[:, :, 0:3] - centroid
    input[:, :, 0:3] = input[:, :, 0:3] / np.expand_dims(furthest_distance, axis=-1)

    input = input[::skip_rate]
    gt = gt[::skip_rate]
    data_radius = data_radius[::skip_rate]
    print("total %d samples" % (len(input)))
    return input, gt, data_radius

### Viz

In [3]:
def viz_pcd_graph(points, edge_list):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(p)
    ls = o3d.geometry.LineSet.create_from_point_cloud_correspondences(pcd, pcd, edge_list)
    ls.paint_uniform_color([0.5, 0.5, 0.5])
    o3d.visualization.draw_geometries([pcd, ls])
    
def viz_many(clouds: list):
    pcds = []
    for i, p in enumerate(clouds):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p + i * np.array([3, 0, 0])) # shift to see them side by side
        pcds.append(pcd)
    o3d.visualization.draw_geometries(pcds)

### Configs

In [8]:
@dataclasses.dataclass
class ModelConfig:
    num_point: int = 256 # number of points per sample
    up_ratio: int = 4 # upsampling ratio
    dilation: int = 2 # dilation in DenseGCN 
    num_neighbours: int = 20 # num neighbours in DenseGCN 
    n_idgcn_blocks: int = 2 # number of inception dense blocks
    channels: int = 32 # number of channels for gcn
    n_dgcn_blocks: int = 3 # number of DenseGCNBlocks in the DenseGCN
    
    
@dataclasses.dataclass
class TrainConfig:
    batch_size: int = 64
    epochs: int = 10
    optimizer = "adam"
    lr = .001
    beta = .9
    

opts = ModelConfig()
train_config = TrainConfig()

### Losses

In [10]:
def chamfer_distance(p: torch.Tensor, q: torch.Tensor):
    """
        p, q: arrays of shape (num_points, num_dim)
    """
    
    closest_to_q = p[knn(p, q, 1)[1]]  # points in p closest to each point in q
    closest_to_p = q[knn(q, p, 1)[1]] # points in q closest to each point in p
    
    e1 = (p - closest_to_p).pow(2).sum(-1)
    e2 = (q - closest_to_q).pow(2).sum(-1)
    cd = e1.mean() + e2.mean()
    return cd

# Data

In [4]:
data_path = os.path.join("..", "data", "PU1K", "train", "pu1k_poisson_256_poisson_1024_pc_2500_patch50_addpugan.h5")

In [5]:
from torch.utils.data import Dataset, DataLoader

In [6]:
class PCDDataset(Dataset):
    def __init__(self, data_path, opts, skip_rate = 1):
        f = h5py.File(data_path, 'r')
        data, ground_truth, data_radius = load_h5_data(h5_filename=data_path, opts=opts, skip_rate = skip_rate, use_randominput=False)
        self.data = torch.tensor(data)
        self.ground_truth = torch.tensor(ground_truth)

    def __getitem__(self, idx):
        return self.data[idx], self.ground_truth[idx]

        
    def __len__(self):
        return len(self.data)
    

In [9]:
dataset = PCDDataset(data_path, opts, 5)

h5_filename :  ..\data\PU1K\train\pu1k_poisson_256_poisson_1024_pc_2500_patch50_addpugan.h5
Do not randominput, input h5 file is: ..\data\PU1K\train\pu1k_poisson_256_poisson_1024_pc_2500_patch50_addpugan.h5
Normalization the data
total 13800 samples


In [22]:
data_cloud, gt_cloud = next(iter(dataset))

In [25]:
viz_many([data_cloud, gt_cloud])

# Training

## Train one point cloud at a time

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
from pugcn_lib import PUGCN

In [13]:
dataset = PCDDataset(data_path, opts, 10)

h5_filename :  ..\data\PU1K\train\pu1k_poisson_256_poisson_1024_pc_2500_patch50_addpugan.h5
Do not randominput, input h5 file is: ..\data\PU1K\train\pu1k_poisson_256_poisson_1024_pc_2500_patch50_addpugan.h5
Normalization the data
total 6900 samples


In [14]:
train_perc = .8
trainset, valset = torch.utils.data.random_split(dataset , [int(len(dataset) * train_perc), int(len(dataset) * (1-train_perc)+1)])

In [15]:
trainloader = DataLoader(trainset, batch_size=64)
valloader = DataLoader(valset, batch_size=64, shuffle=False)

In [16]:
pugcn = PUGCN(opts).to(device)

loss_fn = chamfer_distance
optimizer = torch.optim.Adam(params = pugcn.parameters(), lr = train_config.lr)

In [17]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [18]:
def train(model, trainset, loss_fn, optimizer):
    
    total_loss = 0.
    for p, q in (t:= tqdm(trainset)):
        p, q = p.to(device), q.to(device)
        optimizer.zero_grad()
        
        pred = model(p)
        loss = loss_fn(pred, q)
        
        loss.backward()
        optimizer.step()
        
        total_loss+=loss.item()
        t.set_description(f"loss = {loss.item() :.2f}")
    return total_loss
    

In [19]:
l = train(pugcn, trainset, loss_fn, optimizer)

  0%|          | 0/5520 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [26]:
pred = pugcn(torch.tensor(data_cloud).to(device)).cpu().detach().numpy()

C:\Users\ZademnW\AppData\Local\Temp/ipykernel_16720/2339331737.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = pugcn(torch.tensor(data_cloud).to(device)).cpu().detach().numpy()


In [27]:
pred

array([[-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       ...,
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128]], dtype=float32)

In [28]:
pred

array([[-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       ...,
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128],
       [-0.02048873,  0.01333656,  0.01265128]], dtype=float32)

In [29]:
viz_many([data_cloud, gt_cloud, pred])